<a href="https://colab.research.google.com/github/RGologorsky/fastmri/blob/master/DeepPit_ROI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive
from google.colab import drive
from pathlib import Path 

ROOT = "/content/drive"    
drive.mount(ROOT)

# Useful paths
PROJ_PATH            = Path(ROOT)/"My Drive/PitProj/"
DICOM_FOLDER         = PROJ_PATH/'fastMRI_brain_DICOM'
MULTICOIL_VAL_FOLDER = PROJ_PATH/'multicoil_val'

Mounted at /content/drive


# Binary Mask ROI

Generate ROI as numpy binary mask from a Slicer segmentation obj

# Data

Data is a paired Nifti file with a Slicer segmentation object.

In [4]:
%%capture

# pip install 
!pip install meshio
!pip install nibabel
!pip install nilearn

In [59]:
# imports
import numpy as np
import nibabel as nib
import nilearn as nil

from nibabel.affines import apply_affine
from scipy.spatial   import ConvexHull, Delaunay

import meshio

# choose nii file + segmentation obj
nii_folder = f"{PROJ_PATH}/play"
nii_file = f"{nii_folder}/ABIDE_50454_MRI_MP-RAGE_br_raw_20120830175600888_S164683_I328693 (1).nii"
seg_file = f"{nii_folder}/Segmentation.obj"

# underling MR info (is LAS+, change to RAS+ coords?)
nii_arr = nib.load(nii_file)
print("NifTi coord system (original):", nib.aff2axcodes(nii_arr.affine), sep=" ")
print("MR shape: ", nii_arr.shape, " (axial, sagittal, coronal)")

NifTi coord system (original): ('L', 'A', 'S')
MR shape:  (160, 480, 512)  (axial, sagittal, coronal)


# Vectorized ROI

Convert Slicer ROI obj ==> numpy binary mask.

Alg:
- inputs:
  - 3d numpy voxel arr reprsenting MR
  - Slicer obj = mesh = a set of points ("a collection of vertices and triangles that define the shape of a polyhedral object")

The set of points in the mesh are given in realworld coords. Hence:
1. Compute triangulation of points in the mesh (the polyehdral obj). 
2. For each voxel in MR (index ijk in the np arr),
  - convert voxel coord to real-world coord
  - check if real-world point is within the polyhedral obj defined by the mesh

Vectorized:
- generate all voxel indices at once (np.indices instead of for loops)
- compute real-world affine transform on all voxel indices at once (matrix multiply)
- compute if real-world location is within polyhedral obj for all points at once

Time savings: >30min => under 15secs

In [90]:
def vectorized_balaji_segmenter(image_path, segmentation_path):

  # import the .nii object
  dicom_img = nib.load(image_path)

  # import the segmentation mesh
  segmentation_mesh = meshio.read(segmentation_path)

  # Compute Delaunay triangulation of points.
  tri = Delaunay(segmentation_mesh.points)

  # define the voxel - realworld mappings 
  voxel_to_realworld_transform = dicom_img.affine
  realworld_to_voxel_transform = np.linalg.inv(voxel_to_realworld_transform)

  # initialize numpy arrays
  dicom_img_numpy_array = np.array(dicom_img.get_fdata())
  binary_segmentation_mask = np.zeros_like(dicom_img_numpy_array, dtype=np.bool_)

  # if you want to spot test a single slice, set the range to "range(80, dicom.shape[0])" this is a slice in the middle of the
  # MRI image which should be segmented. Then, uncomment the show_slices line to see the MRI and the segmentation

  # for readability
  shape0, shape1, shape2 = dicom_img_numpy_array.shape

  # from SO: https://stackoverflow.com/questions/12864445/how-to-convert-the-output-of-meshgrid-to-the-corresponding-array-of-points
  # equiv: np.array([(i,j,k) for i in range(shape0) for j in range(shape1) for k in range(shape2)])
  #voxel_location_array = np.array(np.meshgrid(range(shape0), range(shape1), range(shape2), indexing='ij')).T.reshape(-1, 3)[:,[2,1,0]]
  voxel_location_array = np.indices((shape2, shape1, shape0)).T.reshape(-1,3)[:,[2,1,0]]
  realworld_location_array = apply_affine(voxel_to_realworld_transform, voxel_location_array)
  binary_segmentation_mask = (tri.find_simplex(realworld_location_array) >= 0).reshape(shape0, shape1, shape2)

  return dicom_img_numpy_array, binary_segmentation_mask

# Test

In [91]:
%time rachel_np_arr, rachel_bin_mask = vectorized_balaji_segmenter(nii_file, seg_file)

CPU times: user 12.8 s, sys: 1.16 s, total: 14 s
Wall time: 12.5 s


In [92]:
orig_np_arr = np.load(f"{PROJ_PATH}/dicom_img_numpy_array.npy")
orig_bin_mask = np.load(f"{PROJ_PATH}/binary_segmentation_mask.npy")

print(f"Original np arr shape: {orig_np_arr.shape}, original binary mask shape: {orig_bin_mask.shape}")
print(f"rachel np arr shape: {rachel_np_arr.shape}, rachel binary mask shape: {rachel_bin_mask.shape}")

print(f"Input np arr equal? ", np.array_equal(orig_np_arr, rachel_np_arr))
print(f"Output binary mask equal?" , np.array_equal(orig_bin_mask, rachel_bin_mask))

Original np arr shape: (160, 480, 512), original binary mask shape: (160, 480, 512)
rachel np arr shape: (160, 480, 512), rachel binary mask shape: (160, 480, 512)
Input np arr equal?  True
Output binary mask equal? True
